In [2]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [11]:
RESET_DATA = False

In [4]:
url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00353.htm#state"

In [5]:
webpage = requests.get(url)

In [6]:
# Build a soup
soup = bs(webpage.content)

In [7]:
senatorByAlphabeticalOrder = soup.find(class_="newspaperDisplay_3column")
senators = senatorByAlphabeticalOrder.span
senators.text

'Baldwin (D-WI), Nay\nBarrasso (R-WY), Yea\nBennet (D-CO), Nay\nBlackburn (R-TN), Yea\nBlumenthal (D-CT), Nay\nBlunt (R-MO), Yea\nBooker (D-NJ), Nay\nBoozman (R-AR), Yea\nBraun (R-IN), Yea\nBrown (D-OH), Nay\nBurr (R-NC), Yea\nCantwell (D-WA), Nay\nCapito (R-WV), Yea\nCardin (D-MD), Nay\nCarper (D-DE), Nay\nCasey (D-PA), Nay\nCassidy (R-LA), Yea\nCollins (R-ME), Yea\nCoons (D-DE), Nay\nCornyn (R-TX), Yea\nCortez Masto (D-NV), Nay\nCotton (R-AR), Yea\nCramer (R-ND), Yea\nCrapo (R-ID), Yea\nCruz (R-TX), Yea\nDaines (R-MT), Yea\nDuckworth (D-IL), Nay\nDurbin (D-IL), Nay\nErnst (R-IA), Yea\nFeinstein (D-CA), Nay\nFischer (R-NE), Yea\nGillibrand (D-NY), Nay\nGraham (R-SC), Yea\nGrassley (R-IA), Yea\nHagerty (R-TN), Yea\nHassan (D-NH), Nay\nHawley (R-MO), Yea\nHeinrich (D-NM), Nay\nHickenlooper (D-CO), Nay\nHirono (D-HI), Nay\nHoeven (R-ND), Yea\nHyde-Smith (R-MS), Yea\nInhofe (R-OK), Yea\nJohnson (R-WI), Yea\nKaine (D-VA), Nay\nKelly (D-AZ), Nay\nKennedy (R-LA), Yea\nKing (I-ME), Nay\nKlobu

In [8]:
senators_list = senators.text.split("\n")
senators_list.remove("")
senators_list.remove("")
print(len(senators_list))
print(senators_list)

100
['Baldwin (D-WI), Nay', 'Barrasso (R-WY), Yea', 'Bennet (D-CO), Nay', 'Blackburn (R-TN), Yea', 'Blumenthal (D-CT), Nay', 'Blunt (R-MO), Yea', 'Booker (D-NJ), Nay', 'Boozman (R-AR), Yea', 'Braun (R-IN), Yea', 'Brown (D-OH), Nay', 'Burr (R-NC), Yea', 'Cantwell (D-WA), Nay', 'Capito (R-WV), Yea', 'Cardin (D-MD), Nay', 'Carper (D-DE), Nay', 'Casey (D-PA), Nay', 'Cassidy (R-LA), Yea', 'Collins (R-ME), Yea', 'Coons (D-DE), Nay', 'Cornyn (R-TX), Yea', 'Cortez Masto (D-NV), Nay', 'Cotton (R-AR), Yea', 'Cramer (R-ND), Yea', 'Crapo (R-ID), Yea', 'Cruz (R-TX), Yea', 'Daines (R-MT), Yea', 'Duckworth (D-IL), Nay', 'Durbin (D-IL), Nay', 'Ernst (R-IA), Yea', 'Feinstein (D-CA), Nay', 'Fischer (R-NE), Yea', 'Gillibrand (D-NY), Nay', 'Graham (R-SC), Yea', 'Grassley (R-IA), Yea', 'Hagerty (R-TN), Yea', 'Hassan (D-NH), Nay', 'Hawley (R-MO), Yea', 'Heinrich (D-NM), Nay', 'Hickenlooper (D-CO), Nay', 'Hirono (D-HI), Nay', 'Hoeven (R-ND), Yea', 'Hyde-Smith (R-MS), Yea', 'Inhofe (R-OK), Yea', 'Johnson (R-W

In [9]:
df = pd.DataFrame(map(lambda x: x.split(","), senators_list),columns=["senator_data", "vote"])
df["senator_data"] = df.senator_data.apply(lambda x: x.strip())
df["vote"] = df.vote.apply(lambda x: x.strip())

In [100]:
if RESET_DATA == True:
    csv_results = {}
def store_answers(q_no=None, answer=None, show=False, flush=False):
    csv_results.update({ 'Q'+str(q_no): answer})
    if(show):
        print(csv_results)
    if(flush):
        with open("result.csv","w") as fl:
            # Check the header is not written to the file
            pd.DataFrame(csv_results).to_csv(fl)

In [13]:
df["senator_name"] = df['senator_data'].apply(lambda x: x.split("(")[0].strip())
df["senator_party_location"] = df['senator_data'].apply(lambda x: x.split("(")[1].split(")")[0].strip())
df["senator_party_letter"] = df['senator_party_location'].apply(lambda x: x.split("-")[0].strip())
df["senator_party"] = df['senator_party_letter'].apply(lambda x: "republican" if x == "R" else "democrat" if x == "D" else "independents" if x == "I" else None)
df["senator_state"] = df["senator_party_location"].apply(lambda x: x.split("-")[1].strip())

In [14]:
# df[df.isna().any(axis=1)]
df.vote.unique()

array(['Nay', 'Yea', 'Not Voting'], dtype=object)

In [15]:
# Q1
democrats = df.loc[df["senator_party_letter"] == "D"] # Get the number of democrats
democrats_yea = democrats.loc[democrats["vote"] == "Yea"]
democrats_nay = democrats.loc[democrats["vote"] == "Nay"]
democrats_not_voting = democrats.loc[democrats["vote"] == "Not Voting"]
no_democrats_nay = len(democrats_nay)
no_democrats_yea = len(democrats_yea)
no_democrats_not_voting = len(democrats_not_voting)
store_answers(1, no_democrats_not_voting)

In [16]:
# Q2
republicans = df.loc[df["senator_party_letter"] == "R"] # Get the number of republicans
republicans_yea = republicans.loc[republicans["vote"] == "Yea"]
republicans_nay = republicans.loc[republicans["vote"] == "Nay"]
republicans_not_voting = republicans.loc[republicans["vote"] == "Not Voting"]
no_republicans_nay = len(republicans_nay)
no_republicans_yea = len(republicans_yea)
no_republicans_not_voting = len(republicans_not_voting)
store_answers(2, no_republicans_yea)

In [17]:
# Q3
senator_name_B = df[df['senator_name'].apply(lambda x: x.startswith("B"))] # I'm assuming no ones name starts with 'b'
senator_name_B_yea = senator_name_B[senator_name_B["vote"] == "Yea"]
store_answers(3, len(senator_name_B_yea))

In [50]:
# Q4
senator_yea = df[df['vote'] == "Yea"]
senator_grouped_states_yea = senator_yea.groupby("senator_state")
# senator_grouped_states_yea.head()
state_voted_yea = senator_grouped_states_yea.vote.count()[senator_grouped_states_yea.vote.count() == 2] # Hacky fix, need to check that the max that can be is 2
store_answers(4, len(state_voted_yea))

In [57]:
# Q5
senator_not_voting = df[df['vote'] == "Not Voting"]
senator_grouped_states_not_voting = senator_not_voting.groupby("senator_state")
state_not_voting_at_least_1 = senator_grouped_states_not_voting.vote.count()[senator_grouped_states_not_voting.vote.count() >= 1]
store_answers(5, len(state_not_voting_at_least_1))

In [91]:
# Q6
senator_republican = df[df['senator_party'] == "republican"]
senator_grouped_states_republican = senator_republican.groupby("senator_state")
# Hacky fix, need to check that the max that can be is 2
senator_state_both_republican = senator_grouped_states_republican.senator_party.count()[senator_grouped_states_republican.senator_party.count() == 2]
store_answers(6, len(senator_state_both_republican))

In [73]:
# Q7
senator_replubican_democrat = df[(df['senator_party'] == "republican") | (df['senator_party'] == "democrat")]
# senator_replubican_democrat
senator_grouped_states_rep_dem = senator_replubican_democrat.groupby("senator_state")
# Hacky fix, need to check that the max that can be is 2
senator_state_rep_dem = senator_grouped_states_rep_dem.senator_party.count()[senator_grouped_states_rep_dem.senator_party.count() == 2]
# Also can be total_no_of_states - df[df['senator_party'] == "independents"], as there are only these 3 parties
store_answers(7, len(senator_state_rep_dem))

In [78]:
# Q8
senator_name_W = df[df['senator_name'].apply(lambda x: x.startswith("W"))]
store_answers(8,len(senator_name_W))

In [82]:
# Q9
votes_democrats = df[(df['senator_party'] == "democrat")] 
votes_republicans = df[(df['senator_party'] == "republican")]
votes_ratio = len(votes_democrats)/len(votes_republicans)
print(len(votes_democrats),len(votes_republicans))
store_answers(9, votes_ratio)

48 50


In [83]:
# Q10


In [86]:
# Q11
store_answers(11, no_democrats_yea) # I have already caculated this in Q1

In [89]:
# Q12
senator_nay = df[df['vote'] == "Nay"]
senator_grouped_states_nay = senator_nay.groupby("senator_state")
# senator_grouped_states_nay.head()
state_voted_nay = senator_grouped_states_nay.vote.count()[senator_grouped_states_nay.vote.count() == 2] # Hacky fix, need to check that the max that can be is 2
store_answers(12, len(state_voted_nay))

In [92]:
# Q13
senator_democrat = df[df['senator_party'] == "democrat"]
senator_grouped_states_democrat = senator_democrat.groupby("senator_state")
# Hacky fix, need to check that the max that can be is 2
senator_state_both_democrat = senator_grouped_states_democrat.senator_party.count()[senator_grouped_states_democrat.senator_party.count() == 2]
store_answers(13, len(senator_state_both_democrat))

In [98]:
# Q14
senator_not_voting = df[df['vote'] == "Not Voting"]
store_answers(14, len(senator_not_voting))

In [101]:
# Q15
# Isnt this the same as Q5?
store_answers(15, len(state_not_voting_at_least_1), True)

{'Q1': 0, 'Q2': 49, 'Q3': 6, 'Q4': 21, 'Q5': 1, 'Q6': 22, 'Q7': 48, 'Q8': 6, 'Q9': 0.96, 'Q11': 0, 'Q12': 22, 'Q13': 21, 'Q14': 1, 'Q15': 1}


In [ ]:
# Q16
# Isnt that a relative answer?